# 目標

## 載入區

In [1]:
import pandas, numpy, re, os

In [12]:
def os_path(list) : 
    os_path = os.path.dirname(os.path.dirname(os.getcwd()))
    for i in list : 
        os_path = os.path.join(os_path, i)
    return os_path

## database 合併

In [13]:
df = pandas.read_csv(os_path(["database", "104_database.csv"]))

## 遺漏值處理

In [3]:
df = df.dropna(how="all", axis=0)

## 重複值排除

## area

In [4]:
def area_classfication(area) :
     try : 
          area_district = area[3:]
     except : 
          area_district = numpy.nan
     return area_district

df["area_district"] = df["area"].map(area_classfication)

In [5]:
def area_restore(area) :
     try : 
          area = area[0:3]
     except : 
          area = numpy.nan
     return area

df["area"] = df["area"].map(area_restore)

In [6]:
df = df[~(df["area"].isnull())]

## number_of_appliants

In [7]:
def number_of_applicants_level(number_of_applicants):
    if number_of_applicants == "0~5人應徵":
        return 1
    elif number_of_applicants == "6~10人應徵":
        return 2
    elif number_of_applicants == "11~30人應徵":       
        return 3       
    elif number_of_applicants == "大於30人應徵" : 
        return 4
    else : 
        return numpy.nan



# 使用 map 方法將 "salary" 欄位中的數值轉換為 "高薪" 或 "低薪"
df['number_of_applicants_level'] = df['number_of_applicants'].map(number_of_applicants_level)

## education

In [8]:
def education_specialist(education):
    if education == "專科":
        return 1
    else :
        return 0

    

df['education_specialist'] = df["education"].map(education_specialist)

In [9]:
def education_none(education):
    if education == "學歷不拘":
        return 1
    else :
        return 0

    

df['education_none'] = df["education"].map(education_none)

In [10]:
def education_University(education):
    if education == "大學":
        return 1
    else :
        return 0   

df['education_University'] = df["education"].map(education_University)

In [11]:
def education_high_school(education):
    if education == "高中":
        return 1
    else :
        return 0

df['education_high_school'] = df["education"].map(education_high_school)

In [12]:
def education_low_high_school(education):
    if education == "高中以下":
        return 1
    else :
        return 0

df['education_low_high_school'] = df["education"].map(education_low_high_school)

In [13]:
def education_phd(education):
    if education == "博士":
        return 1
    else :
        return 0

df['education_phd'] = df["education"].map(education_phd)

In [14]:
def education_master(education):
    if education == "碩士":
        return 1
    else :
        return 0

df['education_master'] = df["education"].map(education_master)

## company_size clean

In [15]:
def company_size_clean(company_size):
     if "員工" in str(company_size):
        return company_size
     else :
        return numpy.nan

df["company_size"] = df["company_size"].map(company_size_clean)

In [16]:
def company_size_re(company_size):
     if re.search(r'\d+', str(company_size)) :
        return int(re.search(r'\d+', str(company_size)).group())

df["company_size"] = df["company_size"].map(company_size_re)

## salary

In [17]:

# 將 salary中 變成員工數的部分，複製到公司規模的欄位。
df.loc[df['salary'].str.contains('人'), "company_size"] =df[df['salary'].str.contains('人')]["salary"]

# 將不屬於，以遺漏值取代。
df.loc[~df['salary'].str.contains('元'), "salary"] = numpy.nan

In [18]:
# 透過正則表達式，將薪水數據轉換，分兩類，月薪、年薪類。
# 再分類，最低值、最高值。
# 共三個欄位

def month_salary_classfication(salary) :
     if "月薪" in str(salary) :
        return 1
     else : 
        return 0

def year_salary_classfication(salary) :
     if "年薪" in str(salary) :
        return 1
     else : 
        return 0

def time_salary_classfication(salary) :
     if "時薪" in str(salary) :
        return 1
     else : 
        return 0

def element_salary_classfication(salary) :
     if "論件計酬" in str(salary) :
        return 1
     else : 
        return 0


df["month_salary_classfication"] = df["salary"].map(month_salary_classfication)
df["year_salary_classfication"] = df["salary"].map(year_salary_classfication)
df["time_salary_classfication"] = df["salary"].map(time_salary_classfication)
df["element_salary_classfication"] = df["salary"].map(element_salary_classfication)

In [19]:
# 最大值、最小值欄位設置
def max_salary(salary) : 
     text = str(salary).replace(",","")
     result = re.findall("\d+", text)
     try : 
        return int(result[1])
     except :
        return numpy.nan

def mix_salary(salary) :
     text = str(salary).replace(",","")
     result = re.findall("\d+", text)
     try : 
        return int(result[0])
     except :
        return numpy.nan

df["max_salary"] = df["salary"].map(max_salary)
df["mix_salary"] = df["salary"].map(mix_salary)

## company_area

In [20]:
def company_area(company) :
     text_list = re.findall(r'(?<=公司住址：).*', str(company))
     return text_list[0]

def company_rename(company) :
     text_list = re.findall(r"公司名：(.+?)\n", str(company))
     return text_list[0]

df["company_area"] = df["company_name"].map(company_area)
df["company_name"] = df["company_name"].map(company_rename)

## 職業區分

In [21]:
# 將職業分類，轉換為中文
def Profession_classification_transfor(Profession_classification) : 
    if Profession_classification == 1 : 
        return "經營/人資類"
    elif Profession_classification == 2 : 
        return "行政/總務/法務類"
    elif Profession_classification == 3 : 
        return "財會/金融專業類"
    elif Profession_classification == 4 : 
        return "行銷/企劃/專案管理類"
    elif Profession_classification == 5 : 
        return "客服/門市/業務/貿易類"
    elif Profession_classification == 6 : 
        return "餐飲/旅遊/美容美髮類"
    elif Profession_classification == 7 : 
        return "資訊軟體系統類"
    elif Profession_classification == 8 : 
        return "操作/技術/維修類"
    elif Profession_classification == 9 : 
        return "資材/物流/運輸類"
    elif Profession_classification == 10 : 
        return "營建/製圖類"
    elif Profession_classification == 11 :
        return "傳播藝術/設計類"
    elif Profession_classification == 12 : 
        return "文字/傳媒工作類"
    elif Profession_classification == 13 : 
        return "醫療/服務保健類"
    elif Profession_classification == 14 : 
        return "學術/教育/輔導類" 
    elif Profession_classification == 15 : 
        return "研發相關類"
    elif Profession_classification == 16 : 
        return "生產製造/品管/環衛類"
    elif Profession_classification == 17 : 
        return "軍警消/保全類"
    elif Profession_classification == 18 : 
        return "其他類"
    else :
        return numpy.nan
df["Profession_classification_chinese"] = df["Profession_classification"].map(Profession_classification_transfor)

# 將 "/" 符號，轉換為 "，"
def Profession_classification_change(Profession_classification) : 
    try : 
        Profession_classification = Profession_classification.replace("/", "、")
        return Profession_classification
    except : 
        return numpy.nan
df["Profession_classification_chinese"] = df["Profession_classification_chinese"].map(Profession_classification_change)

## 排除多於縣市

In [22]:
df = df[~(df["area"].isin(["越南", "其他亞", "史瓦濟", "美西地", "日本", "印尼", "新加坡", "墨西哥"]))]

## 字串資料清理

In [23]:
def str_clean(str_variable) :
     str_clean = str(str_variable).replace(" ", "")
     return str_clean

for variable in list(df.columns) :
   if df[variable].dtype == object :
      df[variable] = df[variable].map(str_clean)
   else :
      pass

## 測試區

In [24]:
# df["company_name"].unique()

## 輸出

In [26]:
df.to_csv(os_path(["database_clean", "data_clean.csv"]), index=False)

In [27]:
df_ex = df.sample(2000)

In [28]:
df_ex["area"].unique()

array(['台北市', '高雄市', '台中市', '新竹市', '新北市', '桃園市', '雲林縣', '南投縣', '苗栗縣',
       '宜蘭縣', '嘉義縣', '新竹縣', '基隆市', '屏東縣', '彰化縣', '台南市', '嘉義市', '台東縣',
       '金門縣', '花蓮縣', '澎湖縣'], dtype=object)